In [1]:
!pip install transformers
!pip install datasets
!pip install SentencePiece
!pip install accelerate -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 117.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 4.6 MB/s eta 0:00:00

In [1]:
from datasets import load_dataset
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [2]:
# Load pre-trained Pegasus model and tokenizer
model_name = 'google/pegasus-cnn_dailymail'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

In [9]:
# Load and preprocess the dataset
train_dataset = load_dataset('cnn_dailymail', '3.0.0', split='train[:50]')  # Load a subset of the "cnn_dailymail" dataset

In [10]:
def preprocess_function(examples):
    inputs = tokenizer(examples['article'], truncation=True, padding='longest')
    targets = tokenizer(examples['highlights'], truncation=True, padding='longest')
    inputs['labels'] = targets['input_ids']
    return inputs

In [11]:
train_dataset = train_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [12]:
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=1,
    predict_with_generate=True,
    num_train_epochs=1,
    save_steps=500,
    save_total_limit=3,
    eval_steps=100,
    logging_steps=100,
    logging_dir='./logs',
    learning_rate=1e-5,
    warmup_steps=500,
    weight_decay=0.01,
)

In [13]:
# Create the Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=train_dataset,
)

In [14]:
# Start training
trainer.train()

Step,Training Loss


TrainOutput(global_step=50, training_loss=4.350811462402344, metrics={'train_runtime': 43.0072, 'train_samples_per_second': 1.163, 'train_steps_per_second': 1.163, 'total_flos': 144473220710400.0, 'train_loss': 4.350811462402344, 'epoch': 1.0})

In [15]:
model.save_pretrained("path/to/save/directory")

In [ ]:
from transformers import PegasusForConditionalGeneration

model = PegasusForConditionalGeneration.from_pretrained("path/to/save/directory")

In [ ]:
input_text = ["Your input text goes here"]
input_encoding = tokenizer(input_text, truncation=True, padding="longest", max_length=512, return_tensors="pt")

# Generate predictions
output = model.generate(input_encoding["input_ids"])
predicted_text = tokenizer.batch_decode(output, skip_special_tokens=True)